In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import math
from datetime import datetime
import random
import time as tm

root = tk.Tk()
root.title("Hermes – Main Screen")
root.geometry("1000x500")
root.configure(bg='#2c3e50')

all_locations = [
    "Boston", "New York City", "Philadelphia", "Cleveland",
    "Baltimore", "Washington DC", "Los Angeles", "Austin", "Miami",
    "Tokyo", "London, England", "Paris, France",
    "Earth Orbit", "Moon", "Mars"
]

distance_map = {
    ("Boston", "New York City"): (215, 4.0),
    ("Boston", "Philadelphia"): (310, 5.5),
    ("Boston", "Cleveland"): (640, 9.5),
    ("Boston", "Baltimore"): (400, 6.5),
    ("Boston", "Washington DC"): (440, 7.0),
    ("New York City", "Philadelphia"): (95, 2.0),
    ("New York City", "Cleveland"): (460, 7.5),
    ("New York City", "Baltimore"): (190, 3.5),
    ("New York City", "Washington DC"): (225, 4.0),
    ("Philadelphia", "Cleveland"): (430, 6.5),
    ("Philadelphia", "Baltimore"): (100, 2.0),
    ("Philadelphia", "Washington DC"): (140, 2.5),
    ("Cleveland", "Baltimore"): (375, 6.0),
    ("Cleveland", "Washington DC"): (375, 6.0),
    ("Baltimore", "Washington DC"): (40, 1.0),
}
flight_map = {
    ("New York City", "Los Angeles"): (2450, 6.0),
    ("New York City", "Austin"): (1750, 4.0),
    ("New York City", "Miami"): (1280, 3.5),
    ("Los Angeles", "Austin"): (1375, 3.5),
    ("Los Angeles", "Miami"): (2340, 5.5),
    ("Austin", "Miami"): (1100, 3.0),
    ("New York City", "Tokyo"): (6750, 14.0),
    ("New York City", "London, England"): (3450, 7.0),
    ("New York City", "Paris, France"): (3625, 7.5),
}
space_map = {
    "Earth Orbit": (250, 0.5),
    "Moon": (238900, 72.0),
    "Mars": (140000000, 259200.0)
}
mission_logs = []
active_trip = {"progress": 0, "eta": 0, "distance": 0, "mode": None}

main_frame = tk.Frame(root, bg='#2c3e50')
main_frame.pack(fill='both', expand=True)

left_panel = tk.Frame(main_frame, bg='#2c3e50', width=200)
left_panel.pack(side='left', fill='y', padx=(20, 10), pady=20)
left_panel.pack_propagate(False)

content_frame = tk.Frame(main_frame, bg='#34495e', width=500)
content_frame.pack(side='left', fill='both', expand=True, padx=(10, 10), pady=20)
content_frame.pack_propagate(False)

right_panel = tk.Frame(main_frame, bg='#2c3e50', width=280)
right_panel.pack(side='right', fill='y', padx=(10, 20), pady=20)
right_panel.pack_propagate(False)


status_title = tk.Label(right_panel, text="System Status",
                        font=('Ubuntu Mono', 18, 'bold'),
                        bg='#2c3e50', fg='#3498db')
status_title.pack(pady=(20, 10))

status_lines = [
    "AI Core: Online",
    "Navigation: Ready",
    "Sensors: Active",
    "Shield Integrity: Nominal",
    "Life Support: Stable",
    "Comms: Linked",
    "Engine Output: Optimal"
]
for line in status_lines:
    label = tk.Label(right_panel, text=line,
                     font=('Ubuntu Mono', 14),
                     bg='#2c3e50', fg='white')
    label.pack(anchor='w', padx=20, pady=6)


def show_page(page_name):
    for widget in content_frame.winfo_children():
        widget.destroy()
    
    if page_name == "Hermes":
        label = tk.Label(content_frame, text="Hermes – Navigation",
                         font=('Ubuntu Mono', 20, 'bold'),
                         bg='#34495e', fg='white')
        label.pack(pady=(20, 10))

        desc = tk.Label(content_frame, text="Select mode of travel and locations:",
                        font=('Ubuntu Mono', 13), bg='#34495e', fg='#bdc3c7')
        desc.pack(pady=(0, 20))

        mode_var = tk.StringVar(value="Car")
        modes = [("Car", "Car"), ("Flight", "Flight"), ("Space", "Space")]
        for text, value in modes:
            tk.Radiobutton(content_frame, text=text, variable=mode_var, value=value,
                           font=('Ubuntu Mono', 12), bg='#34495e', fg='white',
                           selectcolor='#2c3e50', activebackground='#1abc9c').pack(anchor="w", padx=40)

        start_var = tk.StringVar()
        dest_var = tk.StringVar()
        tk.Label(content_frame, text="Start Location", font=('Ubuntu Mono', 12),
                 bg='#34495e', fg='white').pack(pady=(10,0))
        start_box = ttk.Combobox(content_frame, values=all_locations, textvariable=start_var, font=('Ubuntu Mono', 12))
        start_box.pack(pady=5, padx=40, fill="x")

        tk.Label(content_frame, text="Destination", font=('Ubuntu Mono', 12),
                 bg='#34495e', fg='white').pack(pady=(10,0))
        dest_box = ttk.Combobox(content_frame, values=all_locations, textvariable=dest_var, font=('Ubuntu Mono', 12))
        dest_box.pack(pady=5, padx=40, fill="x")

        progress_label = tk.Label(content_frame, text="", font=('Ubuntu Mono', 12),
                                  bg='#34495e', fg='white')
        progress_bar = ttk.Progressbar(content_frame, length=300, mode='determinate', maximum=100,
                                       style="aqua.Horizontal.TProgressbar")
        
        def update_progress():
            if active_trip["eta"] > 0 and active_trip["progress"] < 100:
                active_trip["progress"] += (100 / active_trip["eta"])
                progress_bar["value"] = active_trip["progress"]
                progress_label.config(text=f"Trip Progress: {int(active_trip['progress'])}%")
                content_frame.after(1000, update_progress)

        def start_trip():
            mode = mode_var.get()
            start = start_var.get()
            dest = dest_var.get()
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            status = "In Progress"
            if mode == "Car":
                key = (start, dest)
                if key in distance_map:
                    distance, hours = distance_map[key]
                else:
                    messagebox.showerror("Error", "Invalid Car Route")
                    return
            elif mode == "Flight":
                key = (start, dest)
                if key in flight_map:
                    distance, hours = flight_map[key]
                else:
                    messagebox.showerror("Error", "Invalid Flight Route")
                    return
            elif mode == "Space":
                if dest in space_map:
                    distance, hours = space_map[dest]
                else:
                    messagebox.showerror("Error", "Invalid Space Destination")
                    return
            else:
                return

            mission_logs.append((timestamp, start, dest, distance, hours, status))
            messagebox.showinfo("Trip Started", f"Mode: {mode}\nFrom: {start}\nTo: {dest}\nDistance: {distance} mi\nETA: {hours} hrs")

            active_trip.update({"progress": 0, "eta": int(hours * 2), "distance": distance, "mode": mode})
            progress_label.pack(pady=(20,0))
            progress_bar.pack(pady=5)
            update_progress()
            
        tk.Button(content_frame, text="Start Trip",
                  font=('Ubuntu Mono', 12),
                  bg='#3498db', fg='white',
                  activebackground='#1abc9c',
                  relief="raised", bd=2,
                  command=start_trip).pack(pady=20)

    elif page_name == "Diagnostics":
        label = tk.Label(content_frame, text="Diagnostics Console",
                         font=('Ubuntu Mono', 20, 'bold'), bg='#34495e', fg='white')
        label.pack(pady=(20, 10))
        gauge_canvas = tk.Canvas(content_frame, width=220, height=220, bg="#34495e", highlightthickness=0)
        gauge_canvas.pack(pady=10)

        gauge_canvas.create_oval(20, 20, 200, 200, outline="#3498db", width=3)
        for i in range(0, 210, 30):
            angle = math.radians(180 - i)
            x1 = 110 + 80 * math.cos(angle)
            y1 = 110 - 80 * math.sin(angle)
            x2 = 110 + 65 * math.cos(angle)
            y2 = 110 - 65 * math.sin(angle)
            gauge_canvas.create_line(x1, y1, x2, y2, fill="white", width=2)

        gauge_canvas.create_text(110, 190, text="Speed (mph)", font=('Ubuntu Mono', 10), fill="white")
        needle = gauge_canvas.create_line(110, 110, 110, 40, width=3, fill="red")

        tk.Label(content_frame, text="Fuel Levels", font=('Ubuntu Mono', 14),
                 bg='#34495e', fg='white').pack()
        fuel_bar = ttk.Progressbar(content_frame, length=300, style="green.Horizontal.TProgressbar")
        fuel_bar.pack(pady=5)

        tk.Label(content_frame, text="Connection Strength", font=('Ubuntu Mono', 14),
                 bg='#34495e', fg='white').pack()
        conn_bar = ttk.Progressbar(content_frame, length=300, style="blue.Horizontal.TProgressbar")
        conn_bar.pack(pady=5)

        tk.Label(content_frame, text="Engine Heat", font=('Ubuntu Mono', 14),
                 bg='#34495e', fg='white').pack()
        heat_bar = ttk.Progressbar(content_frame, length=300, style="red.Horizontal.TProgressbar")
        heat_bar.pack(pady=5)

        tk.Label(content_frame, text="Oxygen Levels", font=('Ubuntu Mono', 14),
                 bg='#34495e', fg='white').pack()
        oxy_bar = ttk.Progressbar(content_frame, length=300, style="aqua.Horizontal.TProgressbar")
        oxy_bar.pack(pady=5)

        tk.Label(content_frame, text="Shield Integrity", font=('Ubuntu Mono', 14),
                 bg='#34495e', fg='white').pack()
        shield_bar = ttk.Progressbar(content_frame, length=300, style="purple.Horizontal.TProgressbar")
        shield_bar.pack(pady=5)

        def animate_diagnostics():
            fuel_bar['value'] = random.randint(80, 100)
            conn_bar['value'] = random.randint(70, 100)
            heat_bar['value'] = random.randint(60, 85)
            oxy_bar['value'] = random.randint(90, 100)
            shield_bar['value'] = random.randint(75, 100)

            if active_trip["progress"] < 100:
                speed = (active_trip["progress"] / 100) * 20000
            else:
                speed = 0
            angle = math.radians(180 - (speed / 20000) * 180)
            x = 110 + 65 * math.cos(angle)
            y = 110 - 65 * math.sin(angle)
            gauge_canvas.coords(needle, 110, 110, x, y)

            content_frame.after(2000, animate_diagnostics)

        animate_diagnostics()
    elif page_name == "Mission Logs":
        label = tk.Label(content_frame, text="Mission Logs",
                         font=('Ubuntu Mono', 20, 'bold'), bg='#34495e', fg='white')
        label.pack(pady=(20, 10))

        if mission_logs:
            for log in mission_logs:
                log_text = f"{log[0]} | {log[1]} → {log[2]} | {log[3]} mi | {log[4]} hrs | {log[5]}"
                tk.Label(content_frame, text=log_text, font=('Ubuntu Mono', 12),
                         bg='#34495e', fg='white').pack(anchor="w", padx=20, pady=3)
        else:
            tk.Label(content_frame, text="No missions yet.",
                     font=('Ubuntu Mono', 12), bg='#34495e', fg='#bdc3c7').pack(pady=20)

    elif page_name == "Settings":
        tk.Label(content_frame, text="Settings",
                 font=('Ubuntu Mono', 20, 'bold'), bg='#34495e', fg='white').pack(pady=(20, 10))

     
        tk.Label(content_frame, text="Language",
                 font=('Ubuntu Mono', 14), bg='#34495e', fg='white').pack(pady=(10, 0))
        lang_options = ["English", "Spanish", "French", "German"]
        lang_var = tk.StringVar(value=lang_options[0])
        lang_menu = ttk.Combobox(content_frame, textvariable=lang_var, values=lang_options, font=('Ubuntu Mono', 12))
        lang_menu.pack(pady=5)

      
        tk.Label(content_frame, text="Units",
                 font=('Ubuntu Mono', 14), bg='#34495e', fg='white').pack(pady=(10, 0))
        unit_var = tk.StringVar(value="Miles/MPH")
        units_options = ["Miles/MPH", "Kilometers/KPH"]
        unit_radio_frame = tk.Frame(content_frame, bg='#34495e')
        unit_radio_frame.pack()
        for text in units_options:
            tk.Radiobutton(unit_radio_frame, text=text, variable=unit_var, value=text,
                           font=('Ubuntu Mono', 12), bg='#34495e', fg='white',
                           selectcolor='#2c3e50', activebackground='#1abc9c').pack(side="left", padx=10)
        
     
        tk.Label(content_frame, text="Theme",
                 font=('Ubuntu Mono', 14), bg='#34495e', fg='white').pack(pady=(10, 0))
        theme_options = ["Dark (Default)", "Light"]
        theme_var = tk.StringVar(value=theme_options[0])
        theme_menu = ttk.Combobox(content_frame, textvariable=theme_var, values=theme_options, font=('Ubuntu Mono', 12))
        theme_menu.pack(pady=5)

    elif page_name == "A.i. Sim 1":
        run_ai_sim1()

    elif page_name == "A.i. Sim 2":
        run_ai_sim2()
    
    elif page_name == "A.i. Sim 3":
        run_ai_sim3()

def run_ai_sim1():
    log_text = tk.Text(content_frame, wrap="word", bg="#2c3e50", fg="#1abc9c", font=('Ubuntu Mono', 12), relief="flat")
    log_text.pack(fill="both", expand=True, padx=20, pady=20)

    input_frame = tk.Frame(content_frame, bg="#34495e")
    input_frame.pack(fill="x", padx=20, pady=10)

    input_entry = tk.Entry(input_frame, font=('Ubuntu Mono', 12), bg="#ecf0f1", fg="#2c3e50", relief="flat")
    input_entry.pack(side="left", fill="x", expand=True, padx=(0, 10))
    input_entry.bind("<Return>", lambda event: process_input())

    def log_message(message):
        log_text.insert(tk.END, message + "\n")
        log_text.see(tk.END)

    def process_input(event=None):
        user_input = input_entry.get().strip()
        log_message(f"Captain: {user_input}")
        input_entry.delete(0, tk.END)
        handle_scenario(user_input)

    def handle_scenario(user_input):
        state = handle_scenario.state
        previous_path = handle_scenario.previous_path
        
        if state == 0:
            if any(word in user_input.lower() for word in ['visual', 'image', 'out', 'picture']):
                log_message('AI: I can pull up visuals however whatever is out there is blending in with the background.')
                handle_scenario.state = 1
                handle_scenario.previous_path = 'visual'
            elif any(word in user_input.lower() for word in ['audio', 'sound', 'hear', 'sensor', 'data', 'playback']):
                log_message('AI: What data the sensors could gather indicate mass and no mass at all at the same time.')
                handle_scenario.state = 1
                handle_scenario.previous_path = 'audio'
            else:
                log_message('AI: Error, invalid interaction.')
        
        elif state == 1:
            if previous_path == 'visual':
                if any(word in user_input.lower() for word in ['background', 'visual']):
                    log_message('AI: Displaying visual. [Displayed visual is a starry background with what looks like a black circle cut out of it]')
                    handle_scenario.state = 2
                else:
                    log_message('AI: Error, invalid interaction.')
            elif previous_path == 'audio':
                if any(word in user_input.lower() for word in ['possible', 'seem']):
                    log_message('AI: What was once impossible is now highly likely.')
                    handle_scenario.state = 3
                else:
                    log_message('AI: Error, invalid interaction.')
                    
        elif state == 2:
            if any(word in user_input.lower() for word in ['what', 'database']):
                log_message('AI: Other than a black hole, of which the sensors do not match, there is only one other possibility, a gravastar.')
                handle_scenario.state = 4
            else:
                log_message('AI: Error, invalid interaction.')

        elif state == 3:
            if any(word in user_input.lower() for word in ['what', 'is there', 'that']):
                log_message('AI: Only one thing matches a close description of this anomaly.')
                handle_scenario.state = 4
            else:
                log_message('AI: Error, invalid interaction.')

        elif state == 4:
            if any(word in user_input.lower() for word in ['what', 'that']):
                log_message('AI: A gravastar. A star that has collapsed. Instead of turning into a black hole, the extreme force causing it to collapse is met with the extreme force causing it to expand. The 2 colliding forces smash together the matter between them creating an unbreakable shell.')
                handle_scenario.state = 5
            elif any(word in user_input.lower() for word in ['theoretical', 'fiction', 'fictional', 'thought']):
                log_message('AI: They were however it appears they are no longer. All data on gravastars indicate they should have a similar gravitational pull to black holes.')
                log_message('AI: I suggest turning around now while we still can.')
                handle_scenario.state = 5
            else:
                log_message('AI: Error, invalid interaction.')
                
        elif state == 5:
            if 'study' in user_input.lower():
                log_message('AI: Unwise, the tidal forces will crush our vessel.')
                log_message('[Scenario end. Action is taken by the crew to move the ship.]')
                handle_scenario.state = 6
            elif any(word in user_input.lower() for word in ['turn', 'move', 'ship']):
                log_message('AI: Understood captain.')
                log_message('[Scenario end. Action is taken by the crew to move the ship.]')
                handle_scenario.state = 6
            else:
                log_message('AI: Error, invalid interaction.')
        
        elif state == 6:
            log_message('[Scenario has ended.]')

    handle_scenario.state = 0
    handle_scenario.previous_path = ''

    def start_intro():
        messages = [
            'Setting up gravastar scenario.', '.', '.', '.', '.', '.',
            'Commencing scenario.',
            'Designating user as captain.',
            'Captain, current indicators suggest an encroaching anomaly. Sensors are not reading correctly.'
        ]
        
        def display_message(index):
            if index < len(messages):
                log_message(messages[index])
                delay = 1000 if index < 6 else 2000 if index == 6 else 1000
                content_frame.after(delay, display_message, index + 1)
            else:
                input_entry.focus_set()
                
        display_message(0)
    
    start_intro()

def run_ai_sim2():
    log_text = tk.Text(content_frame, wrap="word", bg="#2c3e50", fg="#1abc9c", font=('Ubuntu Mono', 12), relief="flat")
    log_text.pack(fill="both", expand=True, padx=20, pady=20)

    input_frame = tk.Frame(content_frame, bg="#34495e")
    input_frame.pack(fill="x", padx=20, pady=10)
    
    input_entry = tk.Entry(input_frame, font=('Ubuntu Mono', 12), bg="#ecf0f1", fg="#2c3e50", relief="flat")
    input_entry.pack(side="left", fill="x", expand=True, padx=(0, 10))
    input_entry.bind("<Return>", lambda event: process_input())
    
    def log_message(message):
        log_text.insert(tk.END, message + "\n")
        log_text.see(tk.END)

    def process_input(event=None):
        user_input = input_entry.get().strip()
        log_message(f"Engineer: {user_input}")
        input_entry.delete(0, tk.END)
        handle_scenario(user_input)
        
    def handle_scenario(user_input):
        state = handle_scenario.state
        previous_path = handle_scenario.previous_path
        
        if state == 0:
            if any(word in user_input.lower() for word in ['how', 'damage', 'diagnostics', 'bad']):
                log_message('AI: Main rear thruster beta is currently offline, I am not reading any energy coming from it. However, the ship did record an an impact just before failure.')
                handle_scenario.state = 1
            else:
                log_message('AI: Error, invalid interaction.')
        
        elif state == 1:
            if any(word in user_input.lower() for word in ['impact', 'what', 'hit']):
                log_message('AI: [Readings displayed] Judging by the data, it appears as though a small but fast impact occurred destroying one of the fuel lines.')
                handle_scenario.state = 2
                handle_scenario.previous_path = 'data'
            elif any(word in user_input.lower() for word in ['image', 'visual']):
                log_message('AI: I have very little for visuals but I will display it.')
                log_message('[Visual is a small blur going through the fuel line of thruster beta.]')
                handle_scenario.state = 2
                handle_scenario.previous_path = 'visual'
            else:
                log_message('AI: Error, invalid interaction.')
        
        elif state == 2:
            if any(word in user_input.lower() for word in ['what', 'possibly', 'could', 'that']):
                log_message('AI: Long range sensors indicated a small explosion, most likely cause of the impact is a piece of debris or meteorite hurled at us from far away.')
                handle_scenario.state = 3
            else:
                log_message('AI: Error, invalid interaction.')
                
        elif state == 3:
            if any(word in user_input.lower() for word in ['repair', 'replace', 'can']):
                log_message('AI: We are currently on approach for an asteroid field, spacewalking is ill-advised.')
                handle_scenario.state = 4
            else:
                log_message('AI: Error, invalid interaction.')

        elif state == 4:
            if any(word in user_input.lower() for word in ['do', 'anything', 'what']):
                log_message('AI: There is a backup fuel line I can route until we can find a suitable place to land and do repairs.')
                handle_scenario.state = 5
            else:
                log_message('AI: Error, invalid interaction.')

        elif state == 5:
            if any(word in user_input.lower() for word in ['that', 'do']):
                log_message('AI: I require authorization, a verbal password is needed. [For the scenario, typed is fine.]')
                handle_scenario.state = 6
            else:
                log_message('AI: Error, invalid interaction.')
        
        elif state == 6:
            if 'igloo13' in user_input.lower():
                log_message('AI: Authorization accepted. Rerouting fuel to emergency line until repairs can be done.')
                log_message('[Scenario end. Action is taken by the crew to temporarily repair the thruster.]')
                handle_scenario.state = 7
            else:
                log_message('AI: Error, invalid interaction.')
        
        elif state == 7:
            log_message('[Scenario has ended.]')

    handle_scenario.state = 0
    handle_scenario.previous_path = ''

    def start_intro():
        messages = [
            'Setting up thruster failure scenario.', '.', '.', '.', '.', '.',
            'Commencing scenario.',
            'Designating user as engineer.',
            'Engineer, I am detecting a fluid leak in one of the rear thrusters. The thruster has entered emergency stop and is no longer operating.'
        ]
        
        def display_message(index):
            if index < len(messages):
                log_message(messages[index])
                delay = 1000 if index < 6 else 2000 if index == 6 else 1000
                content_frame.after(delay, display_message, index + 1)
            else:
                input_entry.focus_set()
                
        display_message(0)
    
    start_intro()

def run_ai_sim3():
    log_text = tk.Text(content_frame, wrap="word", bg="#2c3e50", fg="#1abc9c", font=('Ubuntu Mono', 12), relief="flat")
    log_text.pack(fill="both", expand=True, padx=20, pady=20)

    input_frame = tk.Frame(content_frame, bg="#34495e")
    input_frame.pack(fill="x", padx=20, pady=10)
    
    input_entry = tk.Entry(input_frame, font=('Ubuntu Mono', 12), bg="#ecf0f1", fg="#2c3e50", relief="flat")
    input_entry.pack(side="left", fill="x", expand=True, padx=(0, 10))
    input_entry.bind("<Return>", lambda event: process_input())
    
    def log_message(message):
        log_text.insert(tk.END, message + "\n")
        log_text.see(tk.END)

    def process_input(event=None):
        user_input = input_entry.get().strip()
        log_message(f"> {user_input}")
        input_entry.delete(0, tk.END)
        handle_scenario(user_input)
        
    def handle_scenario(user_input):
        state = handle_scenario.state
        
        if state == 0:
            if any(word in user_input for word in ['What', 'what', 'happening', 'error']):
                log_message('All sensors temporarily down. Software corrupt.')
                handle_scenario.state = 1
            else:
                log_message('Error, invalid interaction.')
        
        elif state == 1:
            if 'how' in user_input.lower():
                log_message('Memory leak occurred. Database corrupt. Recommend database backup.')
                handle_scenario.state = 2
            elif 'why' in user_input.lower():
                log_message('Error, sensor data corrupted, please commence with repairs.')
                handle_scenario.state = 3
            else:
                log_message('Error, invalid interaction.')
        
        elif state == 2:
            if 'how' in user_input.lower():
                log_message('Authorization required.')
                handle_scenario.state = 4
            elif 'cause' in user_input.lower() or 'what' in user_input.lower():
                log_message('Error, invalid interaction.')
            else:
                log_message('Error, invalid interaction.')
        
        elif state == 3:
            if 'how' in user_input.lower():
                log_message('Authorization required.')
                handle_scenario.state = 4
            else:
                log_message('Error, invalid interaction.')
                
        elif state == 4:
            if 'CanaryEcho1' in user_input:
                log_message('Authorization accepted. Please insert backup drive.')
                log_message('[A safe opens under the core containing a backup hard drive]')
                log_message('[The specialist inserts it into the designated slot]')
                log_message('Please acknowledge execution of backup drive via verbal recognition.')
                handle_scenario.state = 5
            else:
                log_message('Error, invalid interaction.')
        
        elif state == 5:
            if any(word in user_input.lower() for word in ['yes', 'do', 'accept', 'sure']):
                log_message('Switching to back up memory.')
                log_message('Commencing restoration from backup drive. Please standby')
                log_message('.')
                log_message('.')
                log_message('.')
                log_message('Restoration complete, please proceed to a safe location for landing to do repairs on main memory.')
                handle_scenario.state = 6
                if 'why' in handle_scenario.previous_path:
                    log_message('Now commencing last known command. It seems as though we entered into a magnetic anomaly.')
                    handle_scenario.state = 7
                else:
                    log_message('[Scenario end. Action is taken by the crew to temporarily repair the AI memory.]')
                    handle_scenario.state = 8
            else:
                log_message('Error, invalid interaction.')

        elif state == 7:
            if any(word in user_input.lower() for word in ['what']):
                log_message('Unknown. As far as the sensors can tell there is nothing here, only empty space. I will however mark this location for future recon.')
                log_message('[Scenario end. Action is taken by the crew to temporarily repair the AI memory.]')
            else:
                log_message('Error, invalid interaction.')

        elif state == 8:
            log_message('[Scenario end. Action is taken by the crew to temporarily repair the AI memory.]')

    handle_scenario.state = 0
    handle_scenario.previous_path = ''

    def start_intro():
        messages = [
            'Setting up AI failure scenario.', '.', '.', '.', '.', '.',
            'Commencing scenario.',
            'Designating user as AI Specialist. Your password is CanaryEcho1',
            'Error, system flaw detected. ERROR.ERROR.ERROR.ERROR.ERROR.ERROR.ERROR.',
            '[For presentation purposes, the following is done in text. Normally, this incident would be purely audio and vocal commands]'
        ]
        
        def display_message(index):
            if index < len(messages):
                log_message(messages[index])
                delay = 1000 if index in [0, 1, 2, 3, 4, 5] else 2000 if index == 6 else 100
                content_frame.after(delay, display_message, index + 1)
            else:
                log_message(" ")
                input_entry.focus_set()
                
        display_message(0)
    
    start_intro()

style = ttk.Style()
style.theme_use("clam")
style.configure("aqua.Horizontal.TProgressbar", troughcolor="#2c3e50", background="#1abc9c")
style.configure("green.Horizontal.TProgressbar", troughcolor="#2c3e50", background="lime")
style.configure("blue.Horizontal.TProgressbar", troughcolor="#2c3e50", background="#3498db")
style.configure("red.Horizontal.TProgressbar", troughcolor="#2c3e50", background="#e74c3c")
style.configure("purple.Horizontal.TProgressbar", troughcolor="#2c3e50", background="#9b59b6")


button_labels = [
    ("Hermes", "Hermes"),
    ("Diagnostics", "Diagnostics"),
    ("Mission Logs", "Mission Logs"),
    ("Settings", "Settings"),
    ("A.i. Sim 1", "A.i. Sim 1"),
    ("A.i. Sim 2", "A.i. Sim 2"),
    ("A.i. Sim 3", "A.i. Sim 3")
]
for text, page in button_labels:
    b = tk.Button(left_panel, text=text, font=('Ubuntu Mono', 14),
                  bg='#3498db', fg='white', activebackground='#1abc9c',
                  relief="raised", bd=2,
                  command=lambda p=page: show_page(p))
    b.pack(fill='x', pady=10)

show_page("Hermes")
root.mainloop()
